# Scikit-learn pierwsza krew
## oraz też trochę Google Colab

Scikit-learn zapewnia interfejs do wielu algotymów machine learningowych.

Google Colab umożliwia wykonywanie obliczeń w chmurze.

## Co analizujemy?
Analizujemy zbiór danych o statystykach pokemonów \\
link: https://www.kaggle.com/abcsds/pokemon\\
## Co chcemy przewidzieć?
Chcemy określić, czy Pokemon jest legendarny.

## Import

In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import io
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier as MLP
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix

## Wczytanie danych

In [0]:
from google.colab import files
uploaded = files.upload() # zbiór powinnyśmy w tym przypadku mieć na dysku

In [0]:
data = pd.read_csv(io.StringIO(uploaded["Pokemon.csv"].decode('utf-8')))

## Przegląd

In [0]:
data.head()

In [0]:
data.shape

In [0]:
sns.barplot(np.arange(2),data.groupby('Legendary').size()) # jak widać mamy do czynienia z niezrównoważonymi danymi

In [0]:
data.isna().sum() # mamy też całkiem sporo NaNów, ale tylko w jednej kolumnie

In [0]:
data = data.iloc[:,2:]
data.sample(5) #możemy wyrzucić pierwszą indeksującą kolumnę oraz imiona Pokemonów

## Podział danych

In [0]:
X_train, X_test, y_train, y_test = train_test_split(data.values[:,0:-1],data.values[:,-1])

In [0]:
print('Stosunek klas w zbiorze uczącym: %f\n Stosunek klas w zbiorze testowym: %f' %(y_train.sum()/len(y_train), y_test.sum()/len(y_test))) # train_test_split zawsze zachowuje stosunek klas, chyba że postanowimy inaczej

In [0]:
y_train = y_train.astype(int)

## Usuwanie Nanów

In [0]:
# Nany są jedynie w kolumne 1
# zrobimy to łatwo przy użyciu pandasa
X_train = pd.DataFrame(X_train)

X_train.iloc[:,1] = X_train.iloc[:,1].fillna(X_train.iloc[:,1].mode().iloc[0])

X_train.head()

## One hot encoding

In [0]:

X_train = pd.get_dummies(X_train, columns=[0,1])
X_train.head()

## Dobór modelu

### Logistic regression

In [0]:
lr_pipe = Pipeline([
    ('stdsc', StandardScaler()),
    ('lda', LDA()),
    ('lr', LogisticRegression())
])

lr_param_grid = [
    {'lda__n_components': [5,10,20,44],
    'lr__C': np.repeat(10.0,5) ** np.arange(-2,3)}
]

lr_gs = GridSearchCV(estimator = lr_pipe,
                    param_grid = lr_param_grid,
                    scoring = 'f1',
                    cv = 2)

lr_scores = cross_val_score(estimator = lr_gs, X_train, y_train.astype(int), scoring = 'f1', cv = 5, n_jobs = -1)

In [0]:
print("To jest f1 dla najlepszego doboru parametrów w każdym z podzbiorów:\n",lr_scores*100)

In [0]:
print("Średni wynik: %.2f +/- %.2f" % (np.mean(lr_scores*100), np.std(lr_scores*100)))

### SVM

In [0]:
svm_pipe = Pipeline([
    ('stdsc', StandardScaler()),
    ('lda', LDA()),
    ('svm', SVC())
])

svm_param_grid = [
    {'lda__n_components': [5,10,20,44],
    'svm__C': np.repeat(10.0,5) ** np.arange(-2,3),
    'svm__gamma': np.repeat(10.0,5) ** np.arange(-2,3),
    'svm__kernel': ['linear', 'rbf']}
]

svm_gs = GridSearchCV(estimator = svm_pipe,
                    param_grid = svm_param_grid,
                    scoring = 'f1',
                    cv = 2)

svm_scores = cross_val_score(svm_gs, X_train, y_train.astype(int), scoring = 'f1', cv = 5, n_jobs = -1)

In [0]:
print("To jest f1 dla najlepszego doboru parametrów w każdym z podzbiorów:\n",svm_scores*100)

In [0]:
print("Średni wynik: %.2f +/- %.2f" % (np.mean(svm_scores*100), np.std(svm_scores*100)))

### Neural network

In [0]:
mlp_pipe = Pipeline([
    ('stdsc', StandardScaler()),
    ('lda', LDA()),
    ('mlp', MLP())
])

mlp_param_grid = [
    {'lda__n_components': [5,10,20,44],
    'mlp__alpha': np.repeat(10.0,5) ** np.arange(-2,3),
    'mlp__hidden_layer_sizes': [(50,50), (100,100), (10,10,10,10)]}
]

mlp_gs = GridSearchCV(estimator = mlp_pipe,
                    param_grid = mlp_param_grid,
                    scoring = 'f1',
                    cv = 2)

mlp_scores = cross_val_score(mlp_gs, X_train, y_train.astype(int), scoring = 'f1', cv = 5, n_jobs = -1)

In [0]:
print("To jest f1 dla najlepszego doboru parametrów w każdym z podzbiorów:\n",mlp_scores*100)

In [0]:
print("Średni wynik: %.2f +/- %.2f" % (np.mean(mlp_scores*100), np.std(mlp_scores*100)))

### RandomTree

In [0]:
rt_pipe = Pipeline([
    ('stdsc', StandardScaler()),
    ('lda', LDA()),
    ('rt', DecisionTreeClassifier())
])

rt_param_grid = [
    {'lda__n_components': [5,10,20,44],
    'rt__max_depth': [3,5,10]}
]

rt_gs = GridSearchCV(estimator = rt_pipe,
                    param_grid = rt_param_grid,
                    scoring = 'f1',
                    cv = 2)

rt_scores = cross_val_score(rt_gs, X_train, y_train.astype(int), scoring = 'f1', cv = 5, n_jobs = -1)

In [0]:
print("To jest f1 dla najlepszego doboru parametrów w każdym z podzbiorów:\n",rt_scores*100)

In [0]:
print("Średni wynik: %.2f +/- %.2f" % (np.mean(rt_scores*100), np.std(rt_scores*100)))

### Random forest

In [0]:
rf_pipe = Pipeline([
    ('stdsc', StandardScaler()),
    ('lda', LDA()),
    ('rf', RandomForestClassifier())
])

rf_param_grid = [
    {'lda__n_components': [5,10,20,44],
    'rf__n_estimators': [10,100,1000],
    'rf__max_depth': [2,3,5, None]}
]

rf_gs = GridSearchCV(estimator = rf_pipe,
                    param_grid = rf_param_grid,
                    scoring = 'f1',
                    cv = 2)

rf_scores = cross_val_score(rf_gs, X_train, y_train.astype(int), scoring = 'f1', cv = 5, n_jobs = -1)

In [0]:
print("To jest f1 dla najlepszego doboru parametrów w każdym z podzbiorów:\n",rf_scores*100)

In [0]:
print("Średni wynik: %.2f +/- %.2f" % (np.mean(rf_scores*100), np.std(rf_scores*100)))

### AdaBoost

In [0]:
ab_pipe = Pipeline([
    ('stdsc', StandardScaler()),
    ('lda', LDA()),
    ('ab', RandomForestClassifier())
])

ab_param_grid = [
    {'lda__n_components': [5,10,20,44],
    'ab__n_estimators': [10,50,100,500]}
]

ab_gs = GridSearchCV(estimator = ab_pipe,
                    param_grid = ab_param_grid,
                    scoring = 'f1',
                    cv = 2)

ab_scores = cross_val_score(ab_gs, X_train, y_train.astype(int), scoring = 'f1', cv = 5, n_jobs = -1)

In [0]:
print("To jest f1 dla najlepszego doboru parametrów w każdym z podzbiorów:\n",ab_scores*100)

In [0]:
print("Średni wynik: %.2f +/- %.2f" % (np.mean(ab_scores*100), np.std(ab_scores*100)))

## Najlepszy klasyfikator

Znaleźliśmy zatem najlepszy klasyfikator do określania czy dany Pokemon jest legendarny czy nie. Okazała się to być las losowy. Można zająć się nim teraz dokładniej.